# Styling complex values in a dataframe

This is the companion notebook of my article [Styling complex values in a dataframe](https://medium.com/p/7ebc5a17b3e7).

In this notebook I show step by step how to use one value in a cell to provide the style for a second value.

## Imports

In [1]:
import pandas as pd
import numpy as np

### Background gradient

Based on numbers, dataframe styles can apply a `background_gradient` based on the values in the dataframe.

In [2]:
# Create a 5x5 matrix of random numbers between [-0.3,0.7)
m = np.random.rand(5,5) - 0.3
df = pd.DataFrame(m)
# shading scale is determined per column
df.style.background_gradient()

,0,1,2,3,4
0,0.697283,0.206823,0.674626,0.206146,-0.091006
1,-0.126214,-0.275965,0.259420,-0.286650,-0.168661
2,0.533885,0.321333,0.340497,-0.216726,0.194268
3,0.612793,0.096373,0.109295,0.327023,0.590566
4,0.033498,0.653014,0.524113,-0.139900,0.624393


## Applying any function to a cell

Next, we apply a method to each cell in the dataframe using `applymap`. We color all negative numbers red.

In [3]:
def style_negative(v, props=''):
    return props if v < 0 else None

df.style.applymap(style_negative, props='color:red;')

,0,1,2,3,4
0,0.697283,0.206823,0.674626,0.206146,-0.091006
1,-0.126214,-0.275965,0.259420,-0.286650,-0.168661
2,0.533885,0.321333,0.340497,-0.216726,0.194268
3,0.612793,0.096373,0.109295,0.327023,0.590566
4,0.033498,0.653014,0.524113,-0.139900,0.624393


## Complex values

Pandas documentation on (applying styles to tables)[https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html] is limited to discussing how the actual values in a dataframe are to be stylised.

But what if we have complex values in our dataframe, tuples, where one element is to be displayed and the other is to determine the stylisation?

In [4]:
import re
# "A Visit from St. Nicholas" by Clement Clarke Moore
# https://www.poetryfoundation.org/poems/43171/a-visit-from-st-nicholas
poem = """
'Twas the night before Christmas, when all through the house
Not a creature was stirring, not even a mouse;
The stockings were hung by the chimney with care,
In hopes that St. Nicholas soon would be there;
"""

words = re.split(r'\W+', poem)
pairs = [(x, len(x)) for x in words]
# Create a 5x5 matrix of tuples: (word, length)
matrix = []
for i in range(0,5):
    matrix.append(pairs[5*i:5*(i+1)])
df2 = pd.DataFrame(matrix)
df2

,0,1,2,3,4
0,"(, 0)","(Twas, 4)","(the, 3)","(night, 5)","(before, 6)"
1,"(Christmas, 9)","(when, 4)","(all, 3)","(through, 7)","(the, 3)"
2,"(house, 5)","(Not, 3)","(a, 1)","(creature, 8)","(was, 3)"
3,"(stirring, 8)","(not, 3)","(even, 4)","(a, 1)","(mouse, 5)"
4,"(The, 3)","(stockings, 9)","(were, 4)","(hung, 4)","(by, 2)"


### Displaying values

To display values we use the `format` method. If the argument to `format` is a callable, then that will be used to convert the value of the cell to what is to be displayed. Here we just want the first element of the tuple in the cell, which is the word.

See: https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.format.html

In [5]:
df2.style.format(lambda x: x[0])

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


### Applying styles to values

We can apply a CSS-styling function to an element by using a style's `applymap` method. Here we use a simple method to determine what colour background is to be used.

See: https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.applymap.html


In [6]:
def font_short(v, weight, color):
    """
    Mark short words
    
    Parameters
    ----------
    v: tuple of (word, length)
    weight: CSS term accepted as a font-weight value
    color: CSS term accepted as a color
    """
    return f"font-weight: {weight}; color: {color}" if v[1] < 4 else None

# We must apply both `format` and `applymap` to the DataFrame.style
styler = df2.style.format(lambda x: x[0])
styler.applymap(font_short, weight='bold', color='orange')

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


### Using a gradient to represent word lengths

Next, let's represent the length of words as a gradient. Long words become dark, short words light. To do this, we must convert the length of the word to a fraction between 0 and 1, relative to the length of the shortest word and the length of the longest word.

We use matplotlib to convert that fraction to a hexadecimal term which we then use to define the background colour in CSS.

In [7]:
import matplotlib as mpl
def make_gradient(v, min_length, max_length, cmap='YlGn'):
    """
    Parameters
    ----------
    
    v: tuple of (word, length)
    min_length: int 
        minimum length of all words in the matrix
    max_length: int
        maximum length of all words in the matrix
    cmap: matplotlib color map, default value here is 'YlGn'
        
    Returns
    -------
    
    string:
        CSS setting a colour
              
    For Matplotlib colormaps:
    See: https://matplotlib.org/stable/tutorials/colors/colormaps.html

    """
    # normalize the word length as a fraction of the range between min_length and max_length
    rel_v = (v[1] - min_length) / (max_length - min_length)
    # define the colormap
    cmap = mpl.cm.get_cmap(cmap)
    # Get a colour out of the given colormap based on a value [0,1]
    rgba = cmap(rel_v)  
    # convert the colour to a hexadecimal string representation
    return f'background-color: {mpl.colors.rgb2hex(rgba)};'

# We must apply both `format` and `applymap` to the DataFrame.style
styler = df2.style.format(lambda x: x[0])
min_length = min([x[1] for x in pairs])
max_length = max([x[1] for x in pairs])
styler.applymap(lambda x: make_gradient(x, min_length, max_length))

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


In [8]:
df_lengths = df2.applymap(lambda x: x[1])
df_words = df2.applymap(lambda x: x[0])
df_words.style.background_gradient(gmap=df_lengths, axis=None)

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by


Now, that was easy! We don't even need to bother with a matrix with tuples to create this background gradient!

### And now with tooltips

So, let's add another functional requirement: colours are nice, but we want to be able to see the exact original word lengths in a tooltip. Let's go back to our matrix of tuples.

We will use the Styler's `set_tooltips` method, which takes as arguments a dataframe and a list of properties. The dataframe should correspond to the dataframe which we want to provide tooltips for, just as df_lengths corresponds to df_words.

We will convert `df_lengths` to a dataframe containing the tooltips for all cells.

In [9]:
df_tooltips = df_lengths.applymap(lambda x: f'length: {x}')
df_tooltips

,0,1,2,3,4
0,length: 0,length: 4,length: 3,length: 5,length: 6
1,length: 9,length: 4,length: 3,length: 7,length: 3
2,length: 5,length: 3,length: 1,length: 8,length: 3
3,length: 8,length: 3,length: 4,length: 1,length: 5
4,length: 3,length: 9,length: 4,length: 4,length: 2


Now we will both apply the gradient using `df_lengths` and the tooltips using `df_tooltips`.

In [10]:
df_lengths = df2.applymap(lambda x: x[1])
df_words = df2.applymap(lambda x: x[0])
styler = df_words.style.background_gradient(gmap=df_lengths, axis=None, cmap='YlOrRd')
styler.set_tooltips(
    df_lengths.applymap(lambda x: f'length: {x}'),
    props=[
        ('visibility', 'hidden'),
        ('position', 'absolute'),
        ('background-color', 'white'),
        ('color', 'black'),
        ('z-index', 1),
        ('padding', '3px 3px'),
        ('margin', '2px')
    ])

,0,1,2,3,4
0,,Twas,the,night,before
1,Christmas,when,all,through,the
2,house,Not,a,creature,was
3,stirring,not,even,a,mouse
4,The,stockings,were,hung,by
